In [1]:
import pickle
import numpy as np
import os

In [2]:
import imageio
import imgaug as ia
import imgaug.augmenters as iaa

In [3]:
# all datasets will be stored in this folder
data_main = "/mnt/ag-microscopy/SampleDataML/OpSeF_XL_Data/"
data_main_run = os.path.join(data_main,"Run")
data_main_GT =  os.path.join(data_main,"GT")

In [4]:
model_dic = {"CP_nuclei":"Cellpose","CP_cyto":"Cellpose",
             "S2D_2D_dsb2018":"StarDist2D","S2D_dsb_versatile":"StarDist2D",
             "S2D_patho":"StarDist2D","S3D_3D_Iso_Basic":"StarDist3D",
             "S3D_3D_Conf_Versatile":"StarDist3D", 
             "UNet_CP001":"Cellprofiler_UNet"}

In [5]:
folder_structure = ["00_InputRaw","01_Input","02_SegMasks","03_SegOverlays","04_BasicQuantification",
                    "05_AdditionalChannel","06_AdditionalMask","07_ClassifiedSegMasks",
                    "08_ClassifiedSegOverlays","09_ClassifiedQuantification","10_ImportExport"]

In [6]:
# define here basic pipelines for augmentation
# see documentation in
# https://github.com/aleju/imgaug-doc/blob/master/index.rst

In [7]:
aug_set = {}

In [8]:
aug_set[0] = iaa.Sequential([
    iaa.Fliplr(1.0),                   # flip image
    iaa.Flipud(1.0),                   # flip image 
    ], random_order=True)

In [9]:
aug_set[1] = iaa.Sequential([
    iaa.Dropout([0.005, 0.02]),      # drop btween 0.5% and 2% of all pixels
    iaa.Sharpen((0.0, 1.0)),        # sharpen the image
    iaa.Fliplr(1.0),                   # flip image 
    iaa.ElasticTransformation(alpha=10, sigma=2)  # apply water effect (affects segmaps)
    ], random_order=True)

In [10]:
aug_set[2] = iaa.Sequential([
    iaa.Dropout([0.005, 0.02]),      # drop btween 0.5% and 2% of all pixels
    iaa.Sharpen((0.0, 1.0)),        # sharpen the image
    iaa.Flipud(1.0),                   # flip image 
    iaa.ElasticTransformation(alpha=50, sigma=5)  # apply water effect (affects segmaps)
    ], random_order=True)

In [11]:
aug_set[3] = iaa.Sequential([
    iaa.Dropout([0.005, 0.02]),      # drop btween 0.5% and 2% of all pixels
    iaa.GaussianBlur((0.0, 2.0)),    # blur the image
    iaa.ElasticTransformation(alpha=20, sigma=3)  # apply water effect (affects segmaps)
    ], random_order=True)

In [12]:
aug_set[4] = iaa.Sequential([
    iaa.GaussianBlur((0.0, 2.0)),     # blur the image
    iaa.GammaContrast((0.5, 2.0)),    # adjust gamma
    iaa.Fliplr(1.0),                   # flip image 
    iaa.ElasticTransformation(alpha=20, sigma=3)  # apply water effect (affects segmaps)
    ], random_order=True)

In [13]:
aug_set[5] = iaa.Sequential([
    iaa.GaussianBlur((0.0, 2.0)),    # blur the image
    iaa.GammaContrast((0.5, 2.0)),   # blur the image
    iaa.Flipud(1.0),                   # flip image 
    iaa.ScaleX((1.0,1.5))            # scale in x (affects segmaps)
    ], random_order=True)

In [14]:
aug_set[6] = iaa.Sequential([
    iaa.GaussianBlur((0.0, 2.0)),    # blur the image
    iaa.GammaContrast((0.5, 2.0)),   # adjust gamma
    iaa.Fliplr(1.0),                # flip image 
    iaa.ScaleX((1.0,1.5)),           # scale in x (affects segmaps)
    iaa.ScaleY((1.0,1.5))            # scale in y (affects segmaps)
    ], random_order=True)

In [15]:
aug_set[7] = iaa.Sequential([
    iaa.imgcorruptlike.SpeckleNoise(severity = 1),   # adds speckle noise
    iaa.GaussianBlur((0.0, 2.0)),    # blur the image
    iaa.GammaContrast((0.5, 2.0)),   # blur the image
    iaa.ScaleX((1.0,1.5)),           # scale in x (affects segmaps)
    iaa.ScaleY((1.0,1.5))            # scale in y (affects segmaps) 
    ], random_order=True)

In [16]:
aug_set[8] = iaa.Sequential([
    iaa.Dropout([0.005, 0.02]),      # drop btween 0.5% and 2% of all pixels
    iaa.GaussianBlur((0.0, 2.0)),    # blur the image
    ], random_order=True)

In [17]:
aug_set[9] = iaa.Sequential([
    iaa.GammaContrast((0.8,1.5)),   # blur the image
    iaa.ScaleX((1.0,1.5)),           # scale in x (affects segmaps)
    iaa.ScaleY((1.0,1.5))            # scale in y (affects segmaps) 
    ], random_order=True)

In [18]:
aug_set[10] = iaa.Sequential([
    iaa.GammaContrast((0.8,1.5)),   # blur the image
    iaa.ScaleX((1.0,1.5)),           # scale in x (affects segmaps)
    iaa.ScaleY((1.0,1.5)),            # scale in y (affects segmaps)
    iaa.Fliplr(1.0),  
    ], random_order=True)

In [19]:
aug_set[11] = iaa.Sequential([
    iaa.Fliplr(1.0),                   # flip image
    iaa.Flipud(1.0),                   # flip image 
    ], random_order=True)

In [20]:
aug_set[12] = iaa.Sequential([
    iaa.Dropout([0.005, 0.05]),      # drop btween 0.5% and 5% of all pixels
    iaa.GammaContrast((0.3, 2.5)),    # adjust gamma
    iaa.Sharpen((0.0, 1.0)),        # sharpen the image
    iaa.Flipud(1.0),                   # flip image 
    iaa.ElasticTransformation(alpha=50, sigma=5)  # apply water effect (affects segmaps)
    ], random_order=True)

In [21]:
aug_set[13] = iaa.Sequential([
    iaa.Dropout([0.005, 0.05]),      # drop btween 0.5% and 2% of all pixels
    iaa.GaussianBlur((0.0, 3.0)),    # blur the image
    iaa.ElasticTransformation(alpha=20, sigma=3)  # apply water effect (affects segmaps)
    ], random_order=True)

In [22]:
aug_set[14] = iaa.Sequential([
    iaa.imgcorruptlike.SpeckleNoise(severity = 2),   # adds speckle noise
    iaa.GaussianBlur((0.0, 3)),    # blur the image
    iaa.GammaContrast((0.5, 3.0)),   # adjust gamma
    iaa.ScaleX((1.0,2.0)),           # scale in x (affects segmaps)
    iaa.ScaleY((1.0,2.0))            # scale in y (affects segmaps) 
    ], random_order=True)

In [23]:
aug_set[15] = iaa.Sequential([
    iaa.Dropout([0.005, 0.05]),      # drop btween 0.5% and 5% of all pixels
    iaa.GammaContrast((0.3, 2.5)),    # adjust gamma
    iaa.Sharpen((0.0, 1.0)),        # sharpen the image
    iaa.imgcorruptlike.Fog((1,3)),
    iaa.Flipud(1.0),                   # flip image 
    iaa.ElasticTransformation(alpha=50, sigma=5)  # apply water effect (affects segmaps)
    ], random_order=True)

In [24]:
aug_set[16] = iaa.Sequential([
    iaa.Dropout([0.005, 0.05]),      # drop btween 0.5% and 5% of all pixels
    iaa.GammaContrast((0.3, 2.5)),    # adjust gamma
    iaa.Sharpen((0.0, 1.0)),        # sharpen the image
    iaa.imgcorruptlike.JpegCompression((1,3)),
    iaa.Flipud(1.0),                   # flip image 
    iaa.ElasticTransformation(alpha=50, sigma=5)  # apply water effect (affects segmaps)
    ], random_order=True)

In [25]:
aug_set[17] = iaa.Sequential([
    iaa.Dropout([0.005, 0.05]),      # drop btween 0.5% and 2% of all pixels
    iaa.GaussianBlur((0.0, 3.0)),    # blur the image
    iaa.imgcorruptlike.ZoomBlur((1)),
    iaa.ElasticTransformation(alpha=20, sigma=3)  # apply water effect (affects segmaps)
    ], random_order=True)

In [26]:
aug_set[18] = iaa.Sequential([
    iaa.GaussianBlur((0.0, 2.0)),    # blur the image
    iaa.imgcorruptlike.SpeckleNoise(severity = 1),   # adds speckle noise
    iaa.GammaContrast((0.5, 2.0)),   # blur the image
    iaa.ScaleX((1.0,1.5)),           # scale in x (affects segmaps)
    iaa.ScaleY((1.0,1.5))            # scale in y (affects segmaps) 
    ], random_order=True)

In [27]:
# here we build out of the basic pipelines entire sets that can easily be used

In [28]:
pre_defined_pipelines = {}

In [29]:
# rather light augmentation
pre_defined_pipelines["Basic_Nuc"] = list(range(0,8))

In [30]:
# light augmentration
pre_defined_pipelines["Light_Nuc"] = [0] + list(range(9,12))

In [31]:
# heavy augmentation, create lot of training images
pre_defined_pipelines["Heavy_Nuc"] = list(range(1,8)) + list(range(12,18))

In [32]:
# mix of all above, create lot of training images 
pre_defined_pipelines["Versatile_Nuc"] = list(range(0,18))

In [33]:
print(pre_defined_pipelines)

{'Basic_Nuc': [0, 1, 2, 3, 4, 5, 6, 7], 'Light_Nuc': [0, 9, 10, 11], 'Heavy_Nuc': [1, 2, 3, 4, 5, 6, 7, 12, 13, 14, 15, 16, 17], 'Versatile_Nuc': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]}


In [34]:
# do no change!
# list of all avaiable datasets for download
Datasets_Download = {}
Datasets_Download["DSB2018_FL_Nuc_Subset"] = "https://owncloud.gwdg.de/index.php/s/mwsVjJKCZMzJjsJ/download"

In [35]:
parameter_main = [model_dic,folder_structure]
parameter_augment = [aug_set,pre_defined_pipelines,data_main_GT,Datasets_Download]

## save parameter

In [36]:
file_name_main = "./Demo_Notebooks/my_runs/main_settings_xl.pkl"
print("Saving main parameter to..",file_name_main)
outfile_main = open(file_name_main,'wb')
pickle.dump(parameter_main,outfile_main)
outfile_main.close()

file_name_augment = "./Train/Augment/my_runs/augment_settings_xl.pkl"
print("Saving augment config parameter to..",file_name_augment)
outfile_aug = open(file_name_augment,'wb')
pickle.dump(parameter_augment,outfile_aug)
outfile_aug.close()



Saving main parameter to.. ./Demo_Notebooks/my_runs/main_settings_xl.pkl
Saving augment config parameter to.. ./Train/Augment/my_runs/augment_settings_xl.pkl
